# Final ETL Load
In this file we will load the resulting csv file to a MySQL database

In [3]:
#We will start by importing all the dependencies
import pymysql
pymysql.install_as_MySQLdb()

from os import path

import pandas as pd

from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base

from config import dbuser, dbpasswd, dburi, dbport, dbname

Base = declarative_base()

In [4]:
#We load the files
file =  pd.read_csv(path.join("..","Clean","final_version","final_database.csv"), delimiter=";")
zonas_patrullaje =  pd.read_csv(path.join("..","Clean","final_version","final_zonas_patrullaje_df.csv"), delimiter=";")
crime_patrol =  pd.read_csv(path.join("..","Clean","final_version","final_crime_patrol_pair.csv"), delimiter=";")



In [5]:
# And transform the SGJ Crime file to a pandas dataframe
db = pd.DataFrame(file)
db.head()

,Año,Mes,Alcaldía,Categoría de delito,Delito,Unidad de investigación,Fecha inicio,Colonia,Geopoint
0,2018,Octubre,ALVARO OBREGON,DELITO DE BAJO IMPACTO,ABUSO DE CONFIANZA,UI-3CD,2018-10-17 13:00:24,LAS AGUILAS 3ER PARQUE,"19.35177196, -99.220583382"
1,2018,Octubre,AZCAPOTZALCO,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,ROBO DE VEHICULO DE SERVICIO PÚBLICO CON VIOLE...,UI-3SD,2018-10-17 13:03:39,EL ROSARIO,"19.5085668561, -99.2083000919"
2,2018,Octubre,GUSTAVO A MADERO,DELITO DE BAJO IMPACTO,ROBO DE OBJETOS,UI-3CD,2018-10-17 13:05:09,GUADALUPE TEPEYAC,"19.4663394581, -99.1243321831"
3,2018,Octubre,GUSTAVO A MADERO,DELITO DE BAJO IMPACTO,ABUSO DE CONFIANZA,UI-3SD,2018-10-17 13:09:04,EMILIANO ZAPATA,"19.4628200485, -99.1163214933"
4,2018,Octubre,MIGUEL HIDALGO,ROBO A REPARTIDOR CON Y SIN VIOLENCIA,ROBO A REPARTIDOR SIN VIOLENCIA,UI-1SD,2018-10-17 13:09:22,PERIODISTA,"19.4503050199, -99.2199589505"


In [6]:
# And transform the Patrol Zones file to a pandas dataframe
zonas_patrullaje_df = pd.DataFrame(zonas_patrullaje)
zonas_patrullaje_df.head()

,Geopoint,Geoshape,Alcaldía,Sector 18,Área km2,x,y,mean,standard_Deviation,distances
0,"19.4559485754, -99.1339187632","{""type"": ""Polygon"", ""coordinates"": [[[-99.1373...",CUAUHTEMOC,TLATELOLCO,0.599031,-99.133437,19.455969,0.527474,0.014291,"[0.534167, 0.437955, 0.262072, 0.536854, 0.648..."
1,"19.4489311584, -99.1492549723","{""type"": ""Polygon"", ""coordinates"": [[[-99.1529...",CUAUHTEMOC,BUENAVISTA,0.542691,-99.149153,19.448988,0.453007,0.006291,"[0.526167, 0.320717, 0.468643, 0.542789, 0.444..."
2,"19.4466167038, -99.1372059309","{""type"": ""Polygon"", ""coordinates"": [[[-99.1388...",CUAUHTEMOC,BUENAVISTA,0.139906,-99.136628,19.446343,0.309541,0.007696,"[0.403358, 0.375808, 0.340076, 0.304342, 0.299..."
3,"19.4345863188, -99.1559474685","{""type"": ""Polygon"", ""coordinates"": [[[-99.1587...",CUAUHTEMOC,REVOLUCION,0.263673,-99.156147,19.434677,0.362777,0.025262,"[0.443341, 0.16611, 0.507736, 0.496656, 0.2699..."
4,"19.4287224255, -99.1566989128","{""type"": ""Polygon"", ""coordinates"": [[[-99.1544...",CUAUHTEMOC,REVOLUCION,0.294612,-99.156832,19.428315,0.378270,0.007552,"[0.469645, 0.426766, 0.407346, 0.198526, 0.457..."


In [7]:
# And transform the Patrol Zones file to a pandas dataframe
crime_patrol_df = pd.DataFrame(crime_patrol)
crime_patrol_df.head()

,crime_index,patrol_index
0,27,0
1,68,0
2,205,0
3,285,0
4,340,0


In [4]:
#We create a class that will represent the delitos table from the MySQL table
class delitos(Base):
    __tablename__="delitos"
    id = Column(Integer, primary_key=True)
    Año = Column(Integer)
    Mes = Column(String)
    Alcaldia = Column(String)
    Categoria_del_delito = Column(String)
    Delito = Column(String)
    Unidad_de_inestigacion = Column(String)
    Fecha_inicio = Column(String)
    Colonia = Column(String)
    Geopoint = Column(String)
    Zona_patrullaje = Column(String)

In [8]:
#We create a class that will represent the zonas_patrullaje table from the MySQL table
class zonas_patrullaje(Base):
    __tablename__="zonas_patrullaje"
    id = Column(Integer, primary_key=True)
    Geopoint = Column(String)
    Geoshape = Column(String)
    Alcaldia = Column(String)
    Sector_18 = Column(String)
    Area = Column(Float)
    x = Column(Float)
    y = Column(Float)
    mean = Column(Float)
    standard_deviation = Column(Float)
    Zona_patrullaje = Column(String)

In [9]:
#We create a class that will represent the crimes_patrol_pair table from the MySQL table
class delitos_zonas_patrullaje(Base):
    __tablename__="delitos_zonas_patrullaje"
    id = Column(Integer, primary_key=True)
    Crime_index = Column(Integer)
    Patrol_index = Column(Integer)    

In [10]:
#Inspect the table we created
Base.metadata.tables

immutabledict({'zonas_patrullaje': Table('zonas_patrullaje', MetaData(bind=None), Column('id', Integer(), table=<zonas_patrullaje>, primary_key=True, nullable=False), Column('Geopoint', String(), table=<zonas_patrullaje>), Column('Geoshape', String(), table=<zonas_patrullaje>), Column('Alcaldia', String(), table=<zonas_patrullaje>), Column('Sector_18', String(), table=<zonas_patrullaje>), Column('Area', Float(), table=<zonas_patrullaje>), Column('x', Float(), table=<zonas_patrullaje>), Column('y', Float(), table=<zonas_patrullaje>), Column('mean', Float(), table=<zonas_patrullaje>), Column('standard_deviation', Float(), table=<zonas_patrullaje>), Column('Zona_patrullaje', String(), table=<zonas_patrullaje>), schema=None), 'delitos_zonas_patrullaje': Table('delitos_zonas_patrullaje', MetaData(bind=None), Column('id', Integer(), table=<delitos_zonas_patrullaje>, primary_key=True, nullable=False), Column('Crime_index', Integer(), table=<delitos_zonas_patrullaje>), Column('Patrol_index', I

In [11]:
#Start engine to our MySQL database
engine = create_engine(f"mysql://{dbuser}:{dbpasswd}@{dburi}:{dbport}/{dbname}")

In [12]:
#Create but first checks if it allready exists
Base.metadata.create_all(engine, checkfirst=True)

In [13]:
#We start the session
from sqlalchemy.orm import Session
session = Session(engine)

# We now begin to insert data
## This is a heavy load and will take considerable amount of time

In [9]:
# There are atleast 250K rows
#We iterate across the local SGJ Crimes dataframe, for each row we add it to the session
for row in db.iterrows() :
    session.add(delitos(Año=row[1][0],Mes=row[1][1],Alcaldia=row[1][2],Categoria_del_delito=row[1][3],
                        Delito=row[1][4],Unidad_de_inestigacion=row[1][5],Fecha_inicio=row[1][6],
                       Colonia=row[1][7],Geopoint=row[1][8],Zona_patrullaje=row[1][9]))
   #print(row[1][0])

In [10]:
#We commit the information to the database
session.commit()

In [ ]:
# There are 698 rows
# We iterate across the local Patrol Zones dataframe, for each row we add it to the session
for row in zonas_patrullaje_df.iterrows():
    session.add(zonas_patrullaje(Geopoint=row[1][0],Geoshape=row[1][1]),Alcaldia=row[1][2],Sector_18=row[1][3],
                Area=row[1][4],x=row[1][5],y=row[1][6],
               mean=row[1][7],standard_deviation=row[1][8],Zona_patrullaje=row[1][9])
    

In [ ]:
session.commit()

In [ ]:
# There are more than 1 million rows (I couldn't count them all before my computer failed LITERALLY) <-----  ¡SUPER IMPORTANT!
# My computer's memory ran out and crashed, therefore I am commiting after each row.
# This IS ineficient, but it will stop the computer from running out of memory.


#We iterate across the local patrol_Crimes_pairs dataframe, for each row we add it to the session
for row in crime_patrol_df.iterrows() :
    session.add(delitos_zonas_patrullaje(Crime_index=row[1][0],Patrol_index=row[1][1]))
    
    #Committing after each row instead of buckets because 1 is the smallest common denominator
    session.commit()
    